# Pair Programming ETL Transformación III

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

## Cargamos el `csv` en el que se trabajará

In [2]:
df = pd.read_csv("files/1.paises_meteo_info.csv", index_col = 0)
df.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea


In [4]:
df.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'pais'],
      dtype='object')

## Definición de variables

In [ ]:
# Definimos las variables que vamos a necesitar para hacer las 
# llamadas a cada uno de los métodos de la clase.

dic_paises = {"USA":[-100.445882,39.7837304,], 
            "Australia":[134.755,-24.7761086],
            "South_Africa":[24.991639,-28.8166236],
            "New Zealand":[172.8344077,-41.5000831],
            "Papua New Guinea":[144.2489081,-5.6816069]}
producto_meteo = "meteo"
producto_astro = "astro"

## Creamos la clase 

In [5]:
class Extraccion: 
    # definimos el método constructor con las variables globales que usaremos.
    def __init__(self):

        # definimos cada una de las variables.
  
    # Este método llamará a la API para cada país

    def seleccionar_paises (diccionario_paises, producto):

        # Definimos la variable producto
        self.df_final = pd.DataFrame()

        # hacemos la llamada  a la API iterando por cada uno de los países
        for k,v in diccionario_paises.items():
            lon = (v[0])
            lat = (v[1])

            url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
        
        response = requests.get(url=url)
        
        codigo_estado = response.status_code
        
        razon_estado = response.reason
        
        if codigo_estado == 200:
            print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        
        response.json()
        # Se convierten los datos obtenidos en un df
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        
        # Creamos una columna llamada país donde se vaya registrando cada país
        df["pais"] = k

        # Concatenamos los df que hemos obtenido con cada país
        df_final = pd.concat([df_final,df],axis=0, ignore_index= True)

        return df_final

    # definimos el método para limpiar los datos obtenidos de la llamada a la API para el producto civil. 
    def limpiar_civil(self, df): 

        # lo primero que tenemos que hacer es crear la columna de fecha en el dataframe nuevo 
        # En este caso será la fecha del día de hoy. 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')

        # creamos la nueva columna
        df["fecha"] = hoy

        # de nuevo usamos el self para crear las nuevas columnas en función de los parámetros pasados al definir la clase. Recordamos, el caminito de baldosas amarillas.
        df["latitud"] = self.lat
        df["longitud"] = self.lon
        df["ciudad"] = self.ciudad
        return df

    # definimos un método para limpiar el resultado de la llamada a la API para el producto astro
    def limpiar_astro(self, df):

        #seleccionamos solo las columnas que nos interesan
        df = df[["seeing", "transparency", "timepoint"]]

        # creamos la columna de fecha: 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        df["fecha"] = hoy


        # insertamos las columnas de la localidad
    
        df["ciudad"] = self.ciudad

        return df
    
    # En este método mergean los df conseguidos. 
    def juntar_dfs(self, df_completo, df_civil, df_visibilidad): 

        df_hoy = pd.merge(df_civil , df_visibilidad , on=['fecha', "timepoint", "ciudad"], how = "right")
        print("El df de hoy es ", df_hoy.columns)
        print("-----------------------------------------")
        print("El df de completo es ", df_completo.columns)
        # lo primero que hacemos es concatenar los dataframes con la información general. df_completo y df_civil
        df_completo = pd.concat([df_completo, df_hoy], axis = 0)

        
        # guardamos los datos
        df_completo.to_pickle('files/datos_actualizados.pkl')
        df_completo.to_csv('files/datos_actualizados.csv')

        return df_completo
    
    
    # Con este método se chequean los datos obtenidos. 
    def chequear_datos(self, df): 
    
        print("Las columnas son:", "\n")
        print(list(df.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df.isnull().sum() / df.shape[0]) *  100)


    # Con este método se limpian las columnas del dataframe, 
   
    def limpiar_dataframe(self, df, lista_columnas = []): 

        #convertimos la fecha a datetime
        df["fecha"] = pd.to_datetime(df["fecha"])

        # reemplazamos los nulos de las columnas por la media
         # lista de columnas en las que queremos reemplazar los nulos
        df[lista_columnas]=df[lista_columnas].fillna(df.mean())

        
        # renombrar columnas
        
        df.rename(columns = {"ciudad_x": "ciudad"}, inplace = True)

        # quitar % 
        df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)

        # guardamos los datos una vez limpios
        df.to_pickle('files/datos_actualizados.pkl')
        df.to_csv('files/datos_actualizados.csv')

        return df

Método para llamar a la API de los países deseados

In [17]:
# Este método llamará a la API para cada país

def seleccionar_paises (diccionario_paises, producto):

    # Definimos la variable producto
    df_final = pd.DataFrame()

    # hacemos la llamada  a la API iterando por cada uno de los países
    for k,v in diccionario_paises.items():
        lon = (v[0])
        lat = (v[1])

        url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
    
    response = requests.get(url=url)
    
    codigo_estado = response.status_code
    
    razon_estado = response.reason
    
    if codigo_estado == 200:
        print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 402:
        print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 404:
        print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    else:
        print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    
    response.json()
    # Se convierten los datos obtenidos en un df
    df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
    
    # Creamos una columna llamada país donde se vaya registrando cada país
    df["pais"] = k

    # Concatenamos los df que hemos obtenido con cada país
    df_final = pd.concat([df_final,df],axis=0, ignore_index= True)

    return df_final


In [18]:
df_paises = seleccionar_paises(dic_paises,producto_meteo)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [19]:
df_paises.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 105, 'speed':...",26,2,10,1010,none,0,0,105,3,Papua New Guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1012,none,1,0,105,3,Papua New Guinea


Método filtrar  

In [ ]:
def filtrar_por_paises (dataframe):

### Aplicación de la clase Extracción

In [8]:
# Creamos una variable llamada api que es resultado de la aplicación de la 
# clase Extracción, recibe como argumentos latitud, longitud y ciudad (Variables definidas anteriormente).

api = Extraccion(latitud, longitud, ciudad)
api